In [ ]:
pip install wildlife-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
pip install wildlife-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (5

In [ ]:
pip install timm

**Importing Wildlife data set and Wildlife tools**



In [ ]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

**Same Data set use for Fine tuning**

**MacaqueFaces Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')

# Load dataset metadata
metadata_MacaqueFaces = datasets.MacaqueFaces('../data/MacaqueFaces')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df, metadata_MacaqueFaces.root, transform=transform)

DATASET MacaqueFaces: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[100:,:], metadata_MacaqueFaces.root, transform=transform)
dataset_query_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[:100,:], metadata_MacaqueFaces.root, transform=transform)

**Importing the pretrain Model**

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_MacaqueFaces = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_MacaqueFaces, database_MacaqueFaces = extractor_MacaqueFaces(dataset_query_MacaqueFaces), extractor_MacaqueFaces(dataset_database_MacaqueFaces)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 49/49 [45:13<00:00, 55.37s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_MacaqueFaces = similarity_function(query_MacaqueFaces, database_MacaqueFaces)
print(similarity_MacaqueFaces)

{'cosine': array([[0.93894947, 0.97903377, 0.9748248 , ..., 0.06067665, 0.0604975 ,
        0.07648014],
       [0.94975156, 0.9809722 , 0.9829178 , ..., 0.08857712, 0.08899386,
        0.10716249],
       [0.9368558 , 0.97414136, 0.9836564 , ..., 0.08185371, 0.08442868,
        0.10064524],
       ...,
       [0.9431046 , 0.985183  , 0.98194647, ..., 0.08535906, 0.08368647,
        0.10288547],
       [0.92794454, 0.98648065, 0.97138005, ..., 0.06948385, 0.06463583,
        0.08578961],
       [0.958699  , 0.9781159 , 0.9700406 , ..., 0.11579873, 0.11533174,
        0.13037327]], dtype=float32)}


In [ ]:
classifier_MacaqueFaces = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
predictions_MacaqueFaces = classifier_MacaqueFaces(similarity_MacaqueFaces['cosine'])
print("Predictions for 100 test Images:-\n",predictions_MacaqueFaces)
accuracy_MacaqueFaces = np.mean(dataset_query_MacaqueFaces.labels_string == predictions_MacaqueFaces)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_MacaqueFaces * 100))
# precision
precision_MacaqueFaces = precision_score(dataset_query_MacaqueFaces.labels_string, predictions_MacaqueFaces, average='weighted')
# recall
recall_MacaqueFaces = recall_score(dataset_query_MacaqueFaces.labels_string, predictions_MacaqueFaces, average='weighted')
# F1 score
f1_MacaqueFaces = f1_score(dataset_query_MacaqueFaces.labels_string, predictions_MacaqueFaces, average='weighted')
print("Precision:", precision_MacaqueFaces)
print("Recall:", recall_MacaqueFaces)
print("F1 Score:", f1_MacaqueFaces)

Predictions for 100 test Images:-
 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan']
Accuracy on MacaqueFaces data: 100.00%
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**LionData Dateset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')

# Load dataset metadata
metadata_LionData = datasets.LionData('../data/LionData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_LionData = WildlifeDataset(metadata_LionData.df, metadata_LionData.root, transform=transform)

DATASET LionData: DOWNLOADING STARTED.


main.zip: 495MB [00:19, 25.3MB/s]


DATASET LionData: EXTRACTING STARTED.
DATASET LionData: FINISHED.



In [ ]:
dataset_database_LionData = WildlifeDataset(metadata_LionData.df.iloc[100:,:], metadata_LionData.root, transform=transform)
dataset_query_LionData = WildlifeDataset(metadata_LionData.df.iloc[:100,:], metadata_LionData.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_LionData = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_LionData, database_LionData = extractor_LionData(dataset_query_LionData), extractor_LionData(dataset_database_LionData)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:38<00:00, 38.28s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 5/5 [03:59<00:00, 47.93s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_LionData = similarity_function(query_LionData, database_LionData)
print(similarity_LionData)

{'cosine': array([[0.86896247, 0.69942033, 0.8777389 , ..., 0.829781  , 0.8954884 ,
        0.86451447],
       [0.9500992 , 0.7055931 , 0.9380013 , ..., 0.7477597 , 0.9478767 ,
        0.89982736],
       [0.87242615, 0.6950849 , 0.89943147, ..., 0.75892425, 0.91286826,
        0.9114601 ],
       ...,
       [0.8723602 , 0.7832219 , 0.9014683 , ..., 0.8094394 , 0.9129671 ,
        0.9049461 ],
       [0.8919123 , 0.73857903, 0.92439425, ..., 0.7845442 , 0.9392824 ,
        0.9575528 ],
       [0.89837366, 0.75758326, 0.91506803, ..., 0.80790067, 0.9342222 ,
        0.93150544]], dtype=float32)}


In [ ]:
# Predictions
classifier_LionData = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
predictions_LionData = classifier_LionData(similarity_LionData['cosine'])
print("Predictions for 100 test Images:-\n",predictions_LionData)
# Accuracy
accuracy_LionData = np.mean(dataset_query_LionData.labels_string == predictions_LionData)
print("Accuracy on Lion data: {:.2f}%".format(accuracy_LionData * 100))
# F1 score,recall and precision
precision = precision_score(dataset_query_LionData.labels_string, predictions_LionData, average='weighted',zero_division=1)
recall = recall_score(dataset_query_LionData.labels_string, predictions_LionData, average='weighted',zero_division=1)
f1 = f1_score(dataset_query_LionData.labels_string, predictions_LionData, average='weighted',zero_division=1)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Predictions for 100 test Images:-
 ['Sebastian' 'Saba' 'Etana' 'Tigisi' 'Sebastian' 'Moswen' 'Naape'
 'Simaloi' 'Autumn' 'Lolparpit' 'Maskio' 'Chaimu' 'Naape' 'Saba' 'Saitoti'
 'Esiriwua' 'Sadala' 'Etana' 'Enderoni' 'Kinna' 'Lucinda' 'Enkume' 'Joy'
 'Lemuanik' 'Naibor' 'Shambe' 'Siti' 'Selenkay' 'Sadala' 'Manzy' 'Moswen'
 'Napi' 'Ngare' 'Naimina' 'Napi' 'Cleopatra' 'Moswen' 'Kioni' 'Chaimu'
 'Engiyaa' 'Enderoni' 'Jamal' 'Leia' 'Sebastian' 'Nuru' 'Maskio' 'Mooza'
 'Moswen' 'Kibibi' 'Tigisi' 'Enderoni' 'Lucinda' 'Moswen' 'Kioni'
 'Sebastian' 'Mashavu' 'Mickey' 'Mama-Kali' 'Kioni' 'Ngare' 'Nashipai'
 'Sikio-Kali' 'Ngare' 'Amber' 'Olonyori' 'Lemuanik' 'Kibibi' 'Ngare'
 'Saimutie' 'Amber' 'Shemsa' 'Olonyori' 'Mickey' 'Esiriwua' 'Nashipai'
 'Enderoni' 'Summer' 'Nuru' 'Mickey' 'Joy' 'Cleopatra' 'Tigisi' 'Mooza'
 'Saitoti' 'Kioni' 'Naini' 'Rescue' 'Sikio-Kali' 'Empurra' 'Enadalut'
 'Lemayian' 'Rescue' 'Enadalut' 'Esiriwua' 'Enkume' 'Senteu' 'Doto'
 'Enkume' 'Nakato' 'Enderoni']
Accuracy on Lio

**NyalaDataSet**

In [ ]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')

# Load dataset metadata
metadata_N = datasets.NyalaData('../data/NyalaData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_N = WildlifeDataset(metadata_N.df, metadata_N.root, transform=transform)

DATASET NyalaData: DOWNLOADING STARTED.


main.zip: 495MB [00:20, 23.9MB/s]


DATASET NyalaData: EXTRACTING STARTED.
DATASET NyalaData: FINISHED.



In [ ]:
dataset_database_N = WildlifeDataset(metadata_N.df.iloc[100:,:], metadata_N.root, transform=transform)
dataset_query_N = WildlifeDataset(metadata_N.df.iloc[:100,:], metadata_N.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_N = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_N, database_N = extractor_N(dataset_query_N), extractor_N(dataset_database_N)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 15/15 [14:08<00:00, 56.59s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_N = similarity_function(query_N, database_N)
print(similarity_N)

{'cosine': array([[0.8619583 , 0.01217471, 0.05642469, ..., 0.6895717 , 0.03031028,
        0.898418  ],
       [0.25952083, 0.918558  , 0.785017  , ..., 0.5010886 , 0.82998675,
        0.1797384 ],
       [0.71705055, 0.4680481 , 0.5621629 , ..., 0.962554  , 0.5190879 ,
        0.7367308 ],
       ...,
       [0.2627344 , 0.9283303 , 0.7531287 , ..., 0.4816041 , 0.80183464,
        0.17906114],
       [0.84421116, 0.0514888 , 0.10151841, ..., 0.7681327 , 0.0811858 ,
        0.9342401 ],
       [0.07699895, 0.7878246 , 0.9208322 , ..., 0.38661018, 0.9427093 ,
        0.07355451]], dtype=float32)}


In [ ]:
# Predictions
classifier_N = KnnClassifier(k=1, database_labels=dataset_database_N.labels_string)
predictions_N = classifier_N(similarity_N['cosine'])
print("Predictions for 100 test Images:-\n",predictions_N)
# Accuracy
accuracy_N = np.mean(dataset_query_N.labels_string == predictions_N)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_N * 100))
# precision,recall and F1 score
precision_N = precision_score(dataset_query_N.labels_string, predictions_N, average='weighted',zero_division=1)
recall_N = recall_score(dataset_query_N.labels_string, predictions_N, average='weighted',zero_division=1)
f1_N = f1_score(dataset_query_N.labels_string, predictions_N, average='weighted',zero_division=1)
print("Precision:", precision_N)
print("Recall:", recall_N)
print("F1 Score:", f1_N)

Predictions for 100 test Images:-
 ['166' '10' '57' '89' '58' '179' '55' '58' '149' '71' '87' '125' '56'
 '144' '131' '8' '36' '150' '131' '11' '116' '19' '20' '86' '260' '165'
 '140' '11' '246' '41' '140' '197' '9' '131' '24' '48' '41' '11' '48'
 '131' '272' '47' '21' '46' '8' '241' '114' '62' '21' '48' '252' '151'
 '27' '108' '82' '51' '101' '85' '32' '58' '46' '139' '63' '85' '9' '181'
 '65' '242' '43' '144' '142' '12' '60' '5' '127' '166' '56' '21' '27' '36'
 '20' '188' '51' '51' '42' '20' '111' '62' '165' '68' '104' '174' '48'
 '217' '144' '6' '50' '91' '105' '140']
Accuracy on NyalaData data: 11.00%
Precision: 0.5458333333333333
Recall: 0.11
F1 Score: 0.11866666666666667


**StripeSpotter Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')
# Load dataset metadata
metadata_S = datasets.StripeSpotter('../data/StripeSpotter')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_S = WildlifeDataset(metadata_S.df, metadata_S.root, transform=transform)

DATASET StripeSpotter: DOWNLOADING STARTED.


data-20110718.zip: 71.7MB [00:00, 81.2MB/s]                           
data-20110718.z02: 100%|██████████| 78.6M/78.6M [00:01<00:00, 64.6MB/s]
data-20110718.z01: 100%|██████████| 78.6M/78.6M [00:00<00:00, 87.7MB/s]


DATASET StripeSpotter: EXTRACTING STARTED.
DATASET StripeSpotter: FINISHED.



In [ ]:
dataset_database_S = WildlifeDataset(metadata_S.df.iloc[100:,:], metadata_S.root, transform=transform)
dataset_query_S = WildlifeDataset(metadata_S.df.iloc[:100,:], metadata_S.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_S = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_S, database_S = extractor_S(dataset_query_S), extractor_S(dataset_database_S)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 6/6 [06:18<00:00, 63.09s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_S = similarity_function(query_S, database_S)
print(similarity_S)

{'cosine': array([[0.2130194 , 0.588757  , 0.65936303, ..., 0.4394258 , 0.44544154,
        0.621335  ],
       [0.17780262, 0.38624054, 0.40102234, ..., 0.7090034 , 0.20244715,
        0.2846234 ],
       [0.6676848 , 0.43850845, 0.34033626, ..., 0.2868997 , 0.39746365,
        0.36011013],
       ...,
       [0.60642976, 0.23701996, 0.21666232, ..., 0.41781956, 0.32831538,
        0.35214955],
       [0.51512223, 0.2798951 , 0.32729602, ..., 0.53868353, 0.39507398,
        0.28750026],
       [0.17924337, 0.24542159, 0.3013066 , ..., 0.58145046, 0.2895743 ,
        0.35858634]], dtype=float32)}


In [ ]:
classifier_S = KnnClassifier(k=1, database_labels=dataset_database_S.labels_string)
predictions_S = classifier_S(similarity_S['cosine'])
print("Predictions for 100 test Images:-\n",predictions_S)
accuracy_S = np.mean(dataset_query_S.labels_string == predictions_S)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_S * 100))
# precision
precision_S = precision_score(dataset_query_S.labels_string, predictions_S, average='weighted',zero_division=1)
# recall
recall_S = recall_score(dataset_query_S.labels_string, predictions_S, average='weighted',zero_division=1)
# F1 score
f1_S = f1_score(dataset_query_S.labels_string, predictions_S, average='weighted',zero_division=1)

print("Precision:", precision_S)
print("Recall:", recall_S)
print("F1 Score:", f1_S)

Predictions for 100 test Images:-
 ['09_212' 'S09_161' '08_085' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' 'S09_067' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '09_212' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '01_661'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 'OP04_225' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679'
 'S09_111' '01_679' '01_036' 'S09_067']
Accuracy on Str

**IPanda50 Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata_P = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_P = WildlifeDataset(metadata_P.df, metadata_P.root, transform=transform)

DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=dd2b004b-4593-424c-83dc-2614a6181d40
To: /data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:12<00:00, 71.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 37.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 157MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



In [ ]:
dataset_database_P = WildlifeDataset(metadata_P.df.iloc[100:,:], metadata_P.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata_P.df.iloc[:100,:], metadata_P.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:39<00:00, 39.98s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 53/53 [45:38<00:00, 51.67s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.02714637,  0.53501487,  0.14056858, ...,  0.38829625,
         0.43481827, -0.07561179],
       [ 0.08373775,  0.5616951 ,  0.03785922, ...,  0.3602667 ,
         0.6493416 , -0.1295084 ],
       [ 0.01190435,  0.4141855 ,  0.23542283, ...,  0.45754877,
         0.5449476 , -0.1335805 ],
       ...,
       [ 0.0320384 ,  0.12872331, -0.14140168, ...,  0.02448   ,
         0.22118227, -0.17661369],
       [-0.08624166,  0.16572046,  0.07199848, ...,  0.14675027,
         0.23473434, -0.02312863],
       [ 0.01022829,  0.06892081,  0.7597883 , ...,  0.4224525 ,
         0.10611828, -0.04714176]], dtype=float32)}


In [ ]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Predictions for 100 test Images:-
 ['10_hexing' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '37_xinger' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '38_xingfan' '04_chengdui' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '24_qixi' '10_hexing' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '04_chengdui' '35_xilan' '08_fulai' '00_aibang' '10_hexing'
 '29_shuangxiong' '11_jiaoao' '44_yayi' '14_maodou' '30_shuqing'
 '36_xingda' '39_xinghui' '33_wuyi' '10_hexing' '10_hexing' '00_aibang'
 '49_yuanrun' '35_xilan' '04_chengdui' '10_hexing' '39_xinghui' '44_yayi'
 '10_hexing' '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui'
 '10_hexing' '17_meibang' '00_aibang' '46_yazhu' '14_maodou' '15_maosun'
 '32_susu' '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomia

CZoo Dataset

In [ ]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')

# Load dataset metadata
metadata_C = datasets.CZoo('../data/CZoo')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_C = WildlifeDataset(metadata_C.df, metadata_C.root, transform=transform)

DATASET CZoo: DOWNLOADING STARTED.


master.zip: 634MB [00:19, 31.8MB/s]


DATASET CZoo: EXTRACTING STARTED.
DATASET CZoo: FINISHED.



In [ ]:
dataset_database_C = WildlifeDataset(metadata_C.df.iloc[100:,:], metadata_C.root, transform=transform)
dataset_query_C = WildlifeDataset(metadata_C.df.iloc[:100,:], metadata_C.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_C = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_C, database_C = extractor_C(dataset_query_C), extractor_C(dataset_database_C)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 16/16 [12:30<00:00, 46.90s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_C = similarity_function(query_C, database_C)
print(similarity_C)

{'cosine': array([[ 9.0549982e-01,  8.9250720e-01,  8.9013600e-01, ...,
        -2.4628341e-02, -1.9136328e-02,  2.0032311e-02],
       [ 9.1913915e-01,  9.1288275e-01,  9.0220594e-01, ...,
        -1.2885875e-02, -3.0940475e-02, -7.5778235e-03],
       [ 9.3326151e-01,  9.2265338e-01,  9.1522133e-01, ...,
        -3.5657752e-02, -1.6588740e-02,  1.8029639e-02],
       ...,
       [ 9.4986129e-01,  9.6489835e-01,  9.4513679e-01, ...,
        -5.2607138e-02, -3.3724420e-03, -1.1571882e-02],
       [ 9.5598102e-01,  9.6259630e-01,  9.4699091e-01, ...,
        -4.5473136e-02, -1.4542699e-02, -3.2175947e-03],
       [ 9.4659024e-01,  9.6087039e-01,  9.4981194e-01, ...,
        -4.7210053e-02, -3.1141117e-03, -8.7950751e-04]], dtype=float32)}


In [ ]:
classifier_C = KnnClassifier(k=1, database_labels=dataset_database_C.labels_string)
predictions_C = classifier_C(similarity_C['cosine'])
print("Predictions for 100 test Images:-\n",predictions_C)
accuracy_C = np.mean(dataset_query_C.labels_string == predictions_C)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_C * 100))
precision_C = precision_score(dataset_query_C.labels_string, predictions_C, average='weighted')
recall_C = recall_score(dataset_query_C.labels_string, predictions_C, average='weighted')
f1_C = f1_score(dataset_query_C.labels_string, predictions_C, average='weighted')
print("Precision:", precision_C)
print("Recall:", recall_C)
print("F1 Score:", f1_C)

Predictions for 100 test Images:-
 ['Alex' 'Alex' 'Alex' 'Jahaga' 'Jahaga' 'Alex' 'Alex' 'Jahaga' 'Alex'
 'Jahaga' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex'
 'Alex' 'Alex' 'Alex' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Gertrudia' 'Gertrudia' 'Gertrudia' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Annett' 'Annett' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Al

In [ ]:
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
conf_matrix = confusion_matrix(dataset_query_C.labels_string, predictions_C)

# Extract elements of confusion matrix
true_positives = np.diag(conf_matrix)
false_positives = np.sum(conf_matrix, axis=0) - true_positives
false_negatives = np.sum(conf_matrix, axis=1) - true_positives
true_negatives = np.sum(conf_matrix) - (true_positives + false_positives + false_negatives)

# Print the results
for i, label in enumerate(np.unique(dataset_query_C.labels_string)):
    print(f"Class: {label}")
    print(f"True Positives: {true_positives[i]}")
    print(f"False Positives: {false_positives[i]}")
    print(f"False Negatives: {false_negatives[i]}")
    print(f"True Negatives: {true_negatives[i]}")


Class: Alex
True Positives: 24
False Positives: 0
False Negatives: 0
True Negatives: 76

Class: Alexandra
True Positives: 67
False Positives: 0
False Negatives: 0
True Negatives: 33

Class: Annett
True Positives: 2
False Positives: 0
False Negatives: 0
True Negatives: 98

Class: Gertrudia
True Positives: 3
False Positives: 0
False Negatives: 0
True Negatives: 97

Class: Jahaga
True Positives: 4
False Positives: 0
False Negatives: 0
True Negatives: 96



**Different Data set for Checking model performance that model never train for it or Unseen data**

**CowDataset**

In [ ]:

# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')

# Load dataset metadata
metadata_COW = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata_COW.df, metadata_COW.root, transform=transform)

DATASET CowDataset: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_COW = WildlifeDataset(metadata_COW.df.iloc[100:,:], metadata_COW.root, transform=transform)
dataset_query_COW = WildlifeDataset(metadata_COW.df.iloc[:100,:], metadata_COW.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_COW = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_COW, database_COW = extractor_COW(dataset_query_COW), extractor_COW(dataset_database_COW)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|████████████████████████████████████████████████████████████████| 1/1 [01:57<00:00, 117.64s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 11/11 [13:04<00:00, 71.34s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_COW = similarity_function(query_COW, database_COW)
print(similarity_COW)

{'cosine': array([[0.39708894, 0.4182203 , 0.5293013 , ..., 0.27663645, 0.4777836 ,
        0.35460728],
       [0.3587119 , 0.4944626 , 0.58756405, ..., 0.31605875, 0.5642501 ,
        0.42149514],
       [0.42812222, 0.4590943 , 0.4853114 , ..., 0.33510396, 0.41814676,
        0.27614745],
       ...,
       [0.41292116, 0.39433795, 0.5616037 , ..., 0.23478842, 0.51470125,
        0.33356407],
       [0.13331117, 0.36302844, 0.23410194, ..., 0.30096102, 0.2174725 ,
        0.347133  ],
       [0.24392274, 0.33747607, 0.43861768, ..., 0.47360572, 0.5489086 ,
        0.6568668 ]], dtype=float32)}


In [ ]:
classifier_COW = KnnClassifier(k=1, database_labels=dataset_database_COW.labels_string)
predictions_COW = classifier_COW(similarity_COW['cosine'])
print("Predictions for 100 test Images:-\n",predictions_COW)
accuracy_COW = np.mean(dataset_query_COW.labels_string == predictions_COW)
print("Accuracy on COW data: {:.2f}%".format(accuracy_COW * 100))
precision_COW = precision_score(dataset_query_COW.labels_string, predictions_COW, average='weighted')
recall_COW = recall_score(dataset_query_COW.labels_string, predictions_COW, average='weighted')
f1_COW = f1_score(dataset_query_COW.labels_string, predictions_COW, average='weighted')
print("Precision:", precision_COW)
print("Recall:", recall_COW)
print("F1 Score:", f1_COW)

Predictions for 100 test Images:-
 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '5' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1' '2'
 '5' '5' '9' '4' '1' '10' '3' '11' '12' '5' '3' '6' '5' '5' '2' '7' '11'
 '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '2' '3' '8' '10'
 '11' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']
Accuracy on COW data: 99.00%
Precision: 0.9907692307692308
Recall: 0.99
F1 Score: 0.9899619047619049


In [ ]:
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
conf_matrix_COW = confusion_matrix(dataset_query_COW.labels_string, predictions_COW)

# Extract elements of confusion matrix
true_positives_COW = np.diag(conf_matrix_COW)
false_positives_COW = np.sum(conf_matrix_COW, axis=0) - true_positives_COW
false_negatives_COW = np.sum(conf_matrix_COW, axis=1) - true_positives_COW
true_negatives_COW = np.sum(conf_matrix_COW) - (true_positives_COW + false_positives_COW + false_negatives_COW)

# Print the results
for i, label in enumerate(np.unique(dataset_query_COW.labels_string)):
    print(f"Class: {label}")
    print(f"True Positives: {true_positives_COW[i]}")
    print(f"False Positives: {false_positives_COW[i]}")
    print(f"False Negatives: {false_negatives_COW[i]}")
    print(f"True Negatives: {true_negatives_COW[i]}")

Class: 1
True Positives: 9
False Positives: 0
False Negatives: 0
True Negatives: 91
Class: 10
True Positives: 6
False Positives: 0
False Negatives: 0
True Negatives: 94
Class: 11
True Positives: 6
False Positives: 0
False Negatives: 0
True Negatives: 94
Class: 12
True Positives: 4
False Positives: 0
False Negatives: 0
True Negatives: 96
Class: 13
True Positives: 4
False Positives: 0
False Negatives: 0
True Negatives: 96
Class: 2
True Positives: 12
False Positives: 1
False Negatives: 0
True Negatives: 87
Class: 3
True Positives: 9
False Positives: 0
False Negatives: 0
True Negatives: 91
Class: 4
True Positives: 8
False Positives: 0
False Negatives: 0
True Negatives: 92
Class: 5
True Positives: 8
False Positives: 0
False Negatives: 0
True Negatives: 92
Class: 6
True Positives: 10
False Positives: 0
False Negatives: 1
True Negatives: 89
Class: 7
True Positives: 7
False Positives: 0
False Negatives: 0
True Negatives: 93
Class: 8
True Positives: 8
False Positives: 0
False Negatives: 0
True 

**DOGFaceNet Dateset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata_D = datasets.DogFaceNet('../data/DogFaceNet')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_D = WildlifeDataset(metadata_D.df, metadata_D.root, transform=transform)

DATASET DogFaceNet: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_D = WildlifeDataset(metadata_D.df.iloc[100:,:], metadata_D.root, transform=transform)
dataset_query_D = WildlifeDataset(metadata_D.df.iloc[:100,:], metadata_D.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_D = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_D, database_D = extractor_D(dataset_query_D), extractor_D(dataset_database_D)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:48<00:00, 48.12s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 65/65 [55:08<00:00, 50.89s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_D = similarity_function(query_D, database_D)
print(similarity_D)

{'cosine': array([[ 0.10996592,  0.13771819,  0.19911267, ...,  0.06574734,
         0.14598583,  0.10803135],
       [ 0.3465269 ,  0.09463353,  0.31827247, ...,  0.07799231,
         0.1425854 ,  0.27729648],
       [ 0.4340247 , -0.02226214,  0.4344272 , ...,  0.08516064,
         0.11263669,  0.39744568],
       ...,
       [ 0.7394483 ,  0.00286073,  0.7671935 , ...,  0.14160475,
         0.113571  ,  0.64189434],
       [ 0.21690877,  0.31127703,  0.20229404, ...,  0.14513196,
         0.11646564,  0.20541766],
       [ 0.04490666,  0.2672245 ,  0.00856524, ...,  0.2642802 ,
         0.1228138 ,  0.05795524]], dtype=float32)}


In [ ]:
classifier_D = KnnClassifier(k=1, database_labels=dataset_database_D.labels_string)
predictions_D = classifier_D(similarity_D['cosine'])
print("Predictions for 100 test Images:-\n",predictions_D)

accuracy_D = np.mean(dataset_query_D.labels_string == predictions_D)
print("Accuracy on DogNet data: {:.2f}%".format(accuracy_D * 100))

precision_D = precision_score(dataset_query_D.labels_string, predictions_D, average='weighted',zero_division=1)
recall_D = recall_score(dataset_query_D.labels_string, predictions_D, average='weighted',zero_division=1)
f1_D = f1_score(dataset_query_D.labels_string, predictions_D, average='weighted',zero_division=1)
print("Precision:", precision_D)
print("Recall:", recall_D)
print("F1 Score:", f1_D)

Predictions for 100 test Images:-
 ['311' '182' '435' '1122' '428' '378' '1063' '923' '222' '405' '764' '190'
 '570' '1339' '432' '1005' '1075' '274' '296' '234' '1104' '991' '247'
 '143' '280' '1156' '223' '1248' '1316' '538' '330' '448' '1336' '251'
 '851' '599' '1082' '210' '345' '563' '1319' '1324' '158' '127' '907'
 '880' '1139' '208' '1241' '424' '293' '172' '99' '835' '373' '192' '366'
 '333' '949' '957' '329' '89' '189' '311' '1003' '631' '490' '646' '26'
 '1368' '148' '722' '174' '1377' '1169' '458' '473' '149' '373' '622'
 '817' '756' '589' '1088' '782' '1357' '868' '462' '1324' '399' '183'
 '200' '1045' '191' '397' '187' '707' '563' '1020' '1074']
Accuracy on DogNet data: 56.00%
Precision: 0.985
Recall: 0.56
F1 Score: 0.57


In [ ]:
# Compute confusion matrix
conf_matrix_D = confusion_matrix(dataset_query_D.labels_string, predictions_D)

# Extract elements of confusion matrix
true_positives_D = np.diag(conf_matrix_D)
false_positives_D = np.sum(conf_matrix_D, axis=0) - true_positives_D
false_negatives_D = np.sum(conf_matrix_D, axis=1) - true_positives_D
true_negatives_D = np.sum(conf_matrix_D) - (true_positives_D + false_positives_D + false_negatives_D)

# Print the results
for i, label in enumerate(np.unique(dataset_query_D.labels_string)):
    print(f"Class: {label}")
    print(f"True Positives: {true_positives_D[i]}")
    print(f"False Positives: {false_positives_D[i]}")
    print(f"False Negatives: {false_negatives_D[i]}")
    print(f"True Negatives: {true_negatives_D[i]}")

Class: 1020
True Positives: 0
False Positives: 1
False Negatives: 0
True Negatives: 99
Class: 1039
True Positives: 0
False Positives: 1
False Negatives: 0
True Negatives: 99
Class: 1063
True Positives: 1
False Positives: 0
False Negatives: 0
True Negatives: 99
Class: 1069
True Positives: 0
False Positives: 0
False Negatives: 1
True Negatives: 99
Class: 1074
True Positives: 0
False Positives: 1
False Negatives: 0
True Negatives: 99
Class: 1075
True Positives: 1
False Positives: 0
False Negatives: 0
True Negatives: 99
Class: 1122
True Positives: 0
False Positives: 0
False Negatives: 1
True Negatives: 99
Class: 1139
True Positives: 1
False Positives: 0
False Negatives: 0
True Negatives: 99
Class: 1169
True Positives: 1
False Positives: 0
False Negatives: 0
True Negatives: 99
Class: 122
True Positives: 0
False Positives: 1
False Negatives: 0
True Negatives: 99
Class: 1232
True Positives: 0
False Positives: 1
False Negatives: 0
True Negatives: 99
Class: 1248
True Positives: 0
False Positive

**MPDD Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.MPDD.get_data('../data/MPDD')

# Load dataset metadata
metadata_M = datasets.MPDD('../data/MPDD')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_M = WildlifeDataset(metadata_M.df, metadata_M.root, transform=transform)

DATASET MPDD: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_M = WildlifeDataset(metadata_M.df.iloc[100:,:], metadata_M.root, transform=transform)
dataset_query_M = WildlifeDataset(metadata_M.df.iloc[:100,:], metadata_M.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_M = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_M, database_M = extractor_M(dataset_query_M), extractor_M(dataset_database_M)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 13/13 [09:33<00:00, 44.11s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_M = similarity_function(query_M, database_M)
print(similarity_M)

{'cosine': array([[ 0.04808867,  0.03224313, -0.00447001, ...,  0.02314246,
         0.02814947,  0.10215738],
       [ 0.16615987,  0.12439519,  0.10770855, ...,  0.14122722,
         0.0507631 ,  0.15300652],
       [ 0.11262   ,  0.05769432,  0.01585272, ...,  0.01326043,
        -0.01141336,  0.07824048],
       ...,
       [ 0.72252524,  0.7501484 ,  0.54229283, ...,  0.30108616,
         0.27135333,  0.41039854],
       [ 0.5749657 ,  0.39892876,  0.5595907 , ...,  0.22507305,
         0.14891459,  0.3073558 ],
       [ 0.58478856,  0.41043365,  0.54034317, ...,  0.34980482,
         0.2591977 ,  0.44655725]], dtype=float32)}


In [ ]:
classifier_M = KnnClassifier(k=1, database_labels=dataset_database_M.labels_string)
predictions_M = classifier_M(similarity_M['cosine'])
print("Predictions for 100 test Images:-\n",predictions_M)

accuracy_M = np.mean(dataset_query_M.labels_string == predictions_M)
print("Accuracy on MPDD data: {:.2f}%".format(accuracy_M * 100))

precision_M = precision_score(dataset_query_M.labels_string, predictions_M, average='weighted',zero_division=1)
recall_M = recall_score(dataset_query_M.labels_string, predictions_M, average='weighted',zero_division=1)
f1_M = f1_score(dataset_query_M.labels_string, predictions_M, average='weighted',zero_division=1)
print("Precision:", precision_M)
print("Recall:", recall_M)
print("F1 Score:", f1_M)

Predictions for 100 test Images:-
 ['1' '1' '152' '1' '114' '1' '1' '2' '70' '152' '28' '76' '76' '76' '2'
 '76' '2' '76' '76' '76' '76' '76' '95' '10' '95' '169' '15' '50' '118'
 '50' '118' '87' '87' '87' '1' '87' '5' '35' '91' '91' '157' '91' '157'
 '5' '91' '157' '25' '145' '145' '145' '145' '145' '145' '145' '145' '145'
 '145' '145' '145' '145' '116' '145' '14' '134' '145' '145' '8' '56' '8'
 '69' '31' '8' '8' '8' '151' '151' '99' '179' '119' '170' '38' '38' '38'
 '178' '38' '38' '62' '62' '38' '38' '14' '38' '14' '117' '14' '14' '135'
 '10' '10' '10']
Accuracy on MPDD data: 3.00%
Precision: 0.9925
Recall: 0.03
F1 Score: 0.02571428571428571


In [ ]:
# Compute confusion matrix
conf_matrix_M = confusion_matrix(dataset_query_M.labels_string, predictions_M)

# Extract elements of confusion matrix
true_positives_M = np.diag(conf_matrix_M)
false_positives_M = np.sum(conf_matrix_M, axis=0) - true_positives_M
false_negatives_M = np.sum(conf_matrix_M, axis=1) - true_positives_M
true_negatives_M = np.sum(conf_matrix_M) - (true_positives_M + false_positives_M + false_negatives_M)

# Print the results
for i, label in enumerate(np.unique(dataset_query_M.labels_string)):
    print(f"Class: {label}")
    print(f"True Positives: {true_positives_M[i]}")
    print(f"False Positives: {false_positives_M[i]}")
    print(f"False Negatives: {false_negatives_M[i]}")
    print(f"True Negatives: {true_negatives_M[i]}")

Class: 0
True Positives: 0
False Positives: 0
False Negatives: 10
True Negatives: 90
Class: 10
True Positives: 0
False Positives: 6
False Negatives: 0
True Negatives: 94
Class: 100
True Positives: 3
False Positives: 1
False Negatives: 0
True Negatives: 96
Class: 101
True Positives: 0
False Positives: 0
False Negatives: 12
True Negatives: 88
Class: 102
True Positives: 0
False Positives: 0
False Negatives: 9
True Negatives: 91
Class: 103
True Positives: 0
False Positives: 0
False Negatives: 7
True Negatives: 93
Class: 104
True Positives: 0
False Positives: 0
False Negatives: 9
True Negatives: 91
Class: 105
True Positives: 0
False Positives: 0
False Negatives: 13
True Negatives: 87
Class: 106
True Positives: 0
False Positives: 0
False Negatives: 6
True Negatives: 94
Class: 107
True Positives: 0
False Positives: 0
False Negatives: 8
True Negatives: 92
Class: 108
True Positives: 0
False Positives: 0
False Negatives: 6
True Negatives: 94
Class: 109
True Positives: 0
False Positives: 0
False 

**PolarBearVidID Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.PolarBearVidID.get_data('../data/PolarBearVidID')

# Load dataset metadata
metadata_PV = datasets.PolarBearVidID('../data/PolarBearVidID')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_PV = WildlifeDataset(metadata_PV.df, metadata_PV.root, transform=transform)

DATASET PolarBearVidID: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_PV = WildlifeDataset(metadata_PV.df.iloc[100:100+4000], metadata_PV.root, transform=transform)
dataset_query_PV = WildlifeDataset(metadata_PV.df.iloc[:100,:], metadata_PV.root, transform=transform)


In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_PV = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_PV, database_PV = extractor_PV(dataset_query_PV), extractor_PV(dataset_database_PV)

100%|███████████████████████████████████████████████████████████████| 32/32 [24:54<00:00, 46.69s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_PV = similarity_function(query_PV, database_PV)
print(similarity_PV)

{'cosine': array([[0.4997698 , 0.4916195 , 0.47195393, ..., 0.44279116, 0.3998226 ,
        0.50805753],
       [0.45085174, 0.44464216, 0.4249237 , ..., 0.4158915 , 0.3735549 ,
        0.45537364],
       [0.49365935, 0.48873666, 0.4689134 , ..., 0.40997854, 0.37677085,
        0.4653929 ],
       ...,
       [0.49911022, 0.49371117, 0.48673368, ..., 0.4957764 , 0.5122148 ,
        0.45701075],
       [0.48214883, 0.47729665, 0.46636096, ..., 0.58403206, 0.59356385,
        0.5038148 ],
       [0.52226436, 0.52092886, 0.5076034 , ..., 0.5275426 , 0.53254277,
        0.47872722]], dtype=float32)}


In [ ]:
classifier_PV = KnnClassifier(k=1, database_labels=dataset_database_PV.labels_string)
predictions_PV = classifier_PV(similarity_PV['cosine'])
print("Predictions for 100 test Images:-\n",predictions_PV)

accuracy_PV = np.mean(dataset_query_PV.labels_string == predictions_PV)
print("Accuracy on PolarBear data: {:.2f}%".format(accuracy_PV * 100))

precision_PV = precision_score(dataset_query_PV.labels_string, predictions_PV, average='weighted',zero_division=1)
recall_PV = recall_score(dataset_query_PV.labels_string, predictions_PV, average='weighted',zero_division=1)
f1_PV = f1_score(dataset_query_PV.labels_string, predictions_PV, average='weighted',zero_division=1)
print("Precision:", precision_PV)
print("Recall:", recall_PV)
print("F1 Score:", f1_PV)

Predictions for 100 test Images:-
 ['Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera'
 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera' 'Vera']
Accuracy on PolarBear data: 100.00%
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
# Compute confusion matrix
conf_matrix_PV = confusion_matrix(dataset_query_PV.labels_string, predictions_PV)

# Extract elements of confusion matrix
true_positives_PV = np.diag(conf_matrix_PV)
false_positives_PV = np.sum(conf_matrix_PV, axis=0) - true_positives_PV
false_negatives_PV = np.sum(conf_matrix_PV, axis=1) - true_positives_PV
true_negatives_PV = np.sum(conf_matrix_PV) - (true_positives_PV + false_positives_PV + false_negatives_PV)

# Print the results
for i, label in enumerate(np.unique(dataset_query_PV.labels_string)):
    print(f"Class: {label}")
    print(f"True Positives: {true_positives_PV[i]}")
    print(f"False Positives: {false_positives_PV[i]}")
    print(f"False Negatives: {false_negatives_PV[i]}")
    print(f"True Negatives: {true_negatives_PV[i]}")

Class: Vera
True Positives: 100
False Positives: 0
False Negatives: 0
True Negatives: 0


**Seen Dataset that not use for fine-tuning**

**OpenCows2020 Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.OpenCows2020.get_data('../data/OpenCows2020')

# Load dataset metadata
metadata_O = datasets.OpenCows2020('../data/OpenCows2020')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_O = WildlifeDataset(metadata_O.df, metadata_O.root, transform=transform)

DATASET OpenCows2020: DOWNLOADING STARTED.


10m32xl88x2b61zlkkgz3fml17.zip: 2.27GB [54:16, 698kB/s]                            


DATASET OpenCows2020: EXTRACTING STARTED.
DATASET OpenCows2020: FINISHED.



In [ ]:
dataset_database_O = WildlifeDataset(metadata_O.df.iloc[100:,:], metadata_O.root, transform=transform)
dataset_query_O = WildlifeDataset(metadata_O.df.iloc[:100,:], metadata_O.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_O = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_O, database_O = extractor_O(dataset_query_O), extractor_O(dataset_database_O)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [01:00<00:00, 60.28s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 37/37 [30:39<00:00, 49.71s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_O = similarity_function(query_O, database_O)
print(similarity_O)

{'cosine': array([[ 3.18159945e-02, -2.34012939e-02,  4.44870442e-04, ...,
        -7.15972260e-02, -2.19623111e-02, -1.56775936e-02],
       [-1.56356879e-02, -2.70779822e-02, -5.03600724e-02, ...,
         2.65440959e-02,  3.45544145e-02,  1.24943638e-02],
       [-1.14652395e-01,  9.86650229e-01,  6.89000636e-02, ...,
         7.33467788e-02, -9.51503217e-02,  2.93384120e-02],
       ...,
       [ 2.60568187e-02, -2.84487624e-02, -3.45171727e-02, ...,
         2.58876719e-02, -2.78105009e-02,  1.63607985e-01],
       [-1.07594673e-02, -4.82171401e-03,  9.37585346e-03, ...,
        -5.25909811e-02, -1.98942348e-02,  4.91162855e-03],
       [-6.81277225e-03, -5.32521307e-03, -4.07319739e-02, ...,
         2.37085149e-02, -3.19451094e-02,  1.54073745e-01]], dtype=float32)}


In [ ]:
classifier_O = KnnClassifier(k=1, database_labels=dataset_database_O.labels_string)
predictions_O = classifier_O(similarity_O['cosine'])
print("Predictions for 100 test Images:-\n",predictions_O)

accuracy_O = np.mean(dataset_query_O.labels_string == predictions_O)
print("Accuracy on PolarBear data: {:.2f}%".format(accuracy_O * 100))

precision_O = precision_score(dataset_query_O.labels_string, predictions_O, average='weighted',zero_division=1)
recall_O = recall_score(dataset_query_O.labels_string, predictions_O, average='weighted',zero_division=1)
f1_O = f1_score(dataset_query_O.labels_string, predictions_O, average='weighted',zero_division=1)
print("Precision:", precision_O)
print("Recall:", recall_O)
print("F1 Score:", f1_O)

Predictions for 100 test Images:-
 ['016' '013' '004' '012' '010' '010' '042' '042' '026' '011' '009' '007'
 '004' '039' '008' '013' '027' '004' '027' '016' '009' '023' '015' '014'
 '007' '015' '001' '026' '002' '018' '004' '015' '012' '018' '045' '019'
 '014' '038' '011' '004' '020' '004' '014' '012' '007' '001' '003' '018'
 '012' '010' '003' '002' '025' '007' '001' '016' '001' '045' '003' '017'
 '004' '012' '015' '005' '039' '014' '015' '006' '042' '004' '001' '015'
 '001' '008' '008' '044' '002' '041' '009' '003' '015' '008' '035' '008'
 '004' '007' '005' '025' '008' '014' '018' '041' '006' '023' '029' '038'
 '001' '006' '016' '006']
Accuracy on PolarBear data: 100.00%
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
# Compute confusion matrix
conf_matrix_O = confusion_matrix(dataset_query_O.labels_string, predictions_O)

# Extract elements of confusion matrix
true_positives_O = np.diag(conf_matrix_O)
false_positives_O = np.sum(conf_matrix_O, axis=0) - true_positives_O
false_negatives_O = np.sum(conf_matrix_O, axis=1) - true_positives_O
true_negatives_O = np.sum(conf_matrix_O) - (true_positives_O + false_positives_O + false_negatives_O)

# Print the results
for i, label in enumerate(np.unique(dataset_query_O.labels_string)):
    print(f"Class: {label}")
    print(f"True Positives: {true_positives_O[i]}")
    print(f"False Positives: {false_positives_O[i]}")
    print(f"False Negatives: {false_negatives_O[i]}")
    print(f"True Negatives: {true_negatives_O[i]}")

Class: 001
True Positives: 7
False Positives: 0
False Negatives: 0
True Negatives: 93
Class: 002
True Positives: 3
False Positives: 0
False Negatives: 0
True Negatives: 97
Class: 003
True Positives: 4
False Positives: 0
False Negatives: 0
True Negatives: 96
Class: 004
True Positives: 9
False Positives: 0
False Negatives: 0
True Negatives: 91
Class: 005
True Positives: 2
False Positives: 0
False Negatives: 0
True Negatives: 98
Class: 006
True Positives: 4
False Positives: 0
False Negatives: 0
True Negatives: 96
Class: 007
True Positives: 5
False Positives: 0
False Negatives: 0
True Negatives: 95
Class: 008
True Positives: 6
False Positives: 0
False Negatives: 0
True Negatives: 94
Class: 009
True Positives: 3
False Positives: 0
False Negatives: 0
True Negatives: 97
Class: 010
True Positives: 3
False Positives: 0
False Negatives: 0
True Negatives: 97
Class: 011
True Positives: 2
False Positives: 0
False Negatives: 0
True Negatives: 98
Class: 012
True Positives: 5
False Positives: 0
False 